# Advent of Code 2023
## Day 24
*<https://adventofcode.com/2023/day/24>*

In [15]:
import heapq
import math
import re
import functools as ft
from collections import Counter, defaultdict, deque, namedtuple
from itertools import combinations, permutations, product
from string import ascii_letters, ascii_lowercase, ascii_uppercase

import IPython
import z3
from rich import inspect, pretty, print

from new_helper import *

pretty.install()

In [16]:
DAY = 24
input_str = get_aoc_input(DAY, 2023)
part_1 = part_2 = 0

In [17]:
inp = input_str.parse_lines()

In [18]:
from typing import NamedTuple


class Hailstone(NamedTuple):
    x: int
    y: int
    z: int
    vx: int
    vy: int
    vz: int


hailstones: list[Hailstone] = []
for line in inp:
    x, y, z, vx, vy, vz = ints(line)
    hailstones.append(Hailstone(x, y, z, vx, vy, vz))

In [19]:
MIN = 200000000000000
MAX = 400000000000000

from fractions import Fraction as Frac


def intersection(a: Hailstone, b: Hailstone) -> tuple[Frac, Frac]:
    return (
        Frac(((a.y - b.y) * b.vx + b.x * b.vy) * a.vx - a.vy * a.x * b.vx, a.vx * b.vy - a.vy * b.vx),
        Frac(((b.x - a.x) * b.vy - b.vx * b.y) * a.vy + a.vx * a.y * b.vy, a.vx * b.vy - a.vy * b.vx),
    )


for a, b in combinations(hailstones, 2):
    try:
        x, y = intersection(a, b)
    except ZeroDivisionError:
        continue

    if (
        MIN <= x <= MAX
        and MIN <= y <= MAX
        and (a.vx >= 0) == (x >= a.x)
        and (a.vy >= 0) == (y >= a.y)
        and (b.vx >= 0) == (x >= b.x)
        and (b.vy >= 0) == (y >= b.y)
    ):
        part_1 += 1

In [20]:
solver = z3.Solver()

rx = z3.Int("rx")
ry = z3.Int("ry")
rz = z3.Int("rz")

rvx = z3.Int("rvx")
rvy = z3.Int("rvy")
rvz = z3.Int("rvz")

for n, h in enumerate(hailstones):
    i = z3.Int(f"t{n}")  # Apparently collisions only happen at integer times

    solver.add(rx + rvx * i == h.x + h.vx * i)
    solver.add(ry + rvy * i == h.y + h.vy * i)
    solver.add(rz + rvz * i == h.z + h.vz * i)

assert solver.check() == z3.sat
part_2 = solver.model().eval(rx + ry + rz)

In [21]:
print_part_1(part_1)
print_part_2(part_2)

Part 1: 20963

Part 2: 999782576459892